## Rainfall & Temp Preprocessing

In [3]:
!pip install pandas 

  Using cached pandas-1.4.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.7 MB)
     |████████████████████████████████| 17.1 MB 4.9 MB/s eta 0:00:01
     |████████████████████████████████| 500 kB 85.8 MB/s eta 0:00:01


In [4]:
import pandas as pd
import glob
import os

In [40]:
root_wd = '/home/studio-lab-user/sagemaker-studiolab-notebooks/'

### Step 1: Glob files together

In [46]:
rainfall_files = glob.glob(os.path.join(root_wd,"data/weather/raw/worldbank_precipitation_1901_2021_africa/pr*.csv"))
rainfall_file_list = []
for f in rainfall_files: 
    df_ = pd.read_csv(f, skiprows=2)
    df_['country_code'] = os.path.basename(f)[36:-4]
    rainfall_file_list.append(df_)
rainfall_df = pd.concat(rainfall_file_list, ignore_index=True)

In [66]:
temp_files = glob.glob(os.path.join(root_wd,"data/weather/raw/worldbank_temp_1901_2021_africa/tas*.csv"))
temp_file_list = []
for f in temp_files: 
    df_ = pd.read_csv(f, skiprows=2)
    df_['country_code'] = os.path.basename(f)[37:-4]
    temp_file_list.append(df_)
temp_df = pd.concat(temp_file_list, ignore_index=True)

In [53]:
temp_df.head(5)

Unnamed: 0    Jan    Feb    Mar    Apr    May    Jun    Jul    Aug    Sep  \
0        1901  19.97  20.08  20.14  20.20  19.84  18.52  19.09  20.09  20.92   
1        1902  19.97  20.09  20.14  20.12  21.33  20.80  19.11  21.28  20.80   
2        1903  19.97  20.15  19.74  20.10  19.85  19.98  19.11  21.28  20.79   
3        1904  19.47  19.51  19.73  19.80  19.97  19.80  19.30  21.27  21.06   
4        1905  19.97  19.91  19.57  19.08  19.61  19.51  18.84  19.94  20.83   

     Oct    Nov    Dec country_code  
0  21.31  20.66  20.77          BDI  
1  21.03  19.76  19.07          BDI  
2  20.72  19.97  19.87          BDI  
3  19.97  19.51  19.39          BDI  
4  21.18  19.85  19.16          BDI

### Step 2: Clean column names

In [54]:
## Map Alpha-3 country code to country name and Alpha-2 country code
africa_country_mapping = pd.read_csv(os.path.join(root_wd,"data/africa_countries.csv"))
mapping_dict = dict(africa_country_mapping[['Alpha-3 code', 'Country']].values)
mapping_dict2 = dict(africa_country_mapping[['Alpha-3 code', 'Alpha-2 code']].values)

In [1]:
mapping_dict

NameError: name 'mapping_dict' is not defined

In [72]:
rainfall_df = rainfall_df.rename(columns={'Unnamed: 0':'Year', 'country_code':'country_cd_alpha3'})
temp_df = temp_df.rename(columns={'Unnamed: 0':'Year','country_code':'country_cd_alpha3'})

rainfall_df['country_name'] = rainfall_df['country_cd_alpha3'].map(mapping_dict)
temp_df['country_name'] = temp_df['country_cd_alpha3'].map(mapping_dict)

rainfall_df['country_cd_alpha2'] = rainfall_df['country_cd_alpha3'].map(mapping_dict2)
temp_df['country_cd_alpha2'] = temp_df['country_cd_alpha3'].map(mapping_dict2)

rainfall_df = rainfall_df.reindex(columns=['country_name','country_cd_alpha3','country_cd_alpha2','Year', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep',
       'Oct', 'Nov', 'Dec'])
temp_df = temp_df.reindex(columns=['country_name','country_cd_alpha3','country_cd_alpha2','Year', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep',
       'Oct', 'Nov', 'Dec'])

rainfall_df = rainfall_df.sort_values(['country_name','Year']).reset_index(drop=True)
temp_df = temp_df.sort_values(['country_name','Year']).reset_index(drop=True)

In [75]:
print(rainfall_df['country_name'].nunique())
print(temp_df['country_name'].nunique())

54
54


In [ ]:
rainfall_df.to_csv(os.path.join(root_wd,"data/africa_countries.csv")), index=False)
temp_df.to_csv('/content/drive/MyDrive/ASDI/rainfall_temp/africa_temp_cleaned.csv', index=False)